In [1]:
# Along-fjord Momentum Balance terms
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import gsw

In [32]:
Qsg = 700
hs = 15
File1 = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf' + str(format(Qsg,'03d')) + '/state_' + str(format(hs,'03d')) + '.nc')
File2 = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf' + str(format(Qsg,'03d')) + '/kpp_state_' + str(format(hs,'03d')) + '.nc')
T0 = 81
Tn = len(File1.T)
State1 = File1.isel(X=range(120), Xp1=range(121), Y=range(30,39), Yp1=range(30,40), T=range(T0,Tn))
State2 = File2.isel(X=range(120), Y=range(30,39), T=range(T0-1,Tn-1))

# U (T,Z,Y,Xp1) W(T,Zl,Y,X) KPPviscAz(T-1,Z,Y,X)
U = State1.U.data.mean(axis=(0,2))
u = (U[:, 1:] + U[:, :-1]) / 2
W0 = State1.W.data.mean(axis=(0,2))
Wbt = np.zeros((1,W0.shape[1])) # Add vertical velocity (0 m/s) at the bottom
W = np.concatenate((W0, Wbt), axis=0)
w = (W[1:, :] + W[:-1, :]) / 2
#W = np.concatenate((W0, Wbt), axis=0)
Av = State2.KPPviscAz.data.mean(axis=(0,2)) # Vetical eddy viscosity (m^2/s)
s = State1.S.data.mean(axis=(0,2)) # Salinity
x = State1.X.data
z = State1.Z.data

pt = State1.Temp.data.mean(axis=(0,2)) # Potential temperature
pres = gsw.p_from_z(z, 0) 
p = np.tile(pres,[len(x),1]).T
#rho = gsw.rho(s, pt, p)



#file2.KPPviscAz
#X, Z = np.meshgrid(x, z)
mask = (s == 0) # Exclude data covered by the sill
uma = np.ma.masked_array(u,mask)
wma = np.ma.masked_array(w,mask)
sma = np.ma.masked_array(s,mask)
Avma = np.ma.masked_array(Av,mask)
#State2.KPPviscAz
pma = np.ma.masked_array(p,mask)
tma = np.ma.masked_array(pt,mask)
rho = gsw.rho(sma, tma, pma)
u[mask] = np.nan


masked_array(data=[--, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --],
             mask=[ True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True],
       fill_value=1e+20,
            dtype=float32)

In [28]:
# Calculate gradient considering the masked values (NaN)
def masked_gradient(Var, X, Z):
    #Var_x = np.full([Var.shape[0], Var.shape[1]], np.nan)
    #Var_z = np.full([Var.shape[0], Var.shape[1]], np.nan)
    Var_x = Var.copy()
    Var_z = Var.copy()

    for i in range(len(Z)):
        if np.array(np.where(np.isnan(Var[i,1:]))).size == 0: # Var[:,0]=NaN for our output
            #xn1[j] = u.shape[1]
            #xn2[j] = u.shape[1]
            Var_x[i,1:] = np.gradient(Var[i,1:],X[1:])
        else:
            xn1 = np.where(np.isnan(Var[i,:]))[-1][1] # Left boundary of the mask
            xn2 = np.where(np.isnan(Var[i,:]))[-1][-1] + 1 # Right boundary of the mask
            Var_x[i,1:xn1] = np.gradient(Var[i,1:xn1], X[1:xn1])
            Var_x[i,xn2:] = np.gradient(Var[i,xn2:], X[xn2:])
    
    for j in range(len(X)-1): # Exclude Var[:,0]
        if np.array(np.where(np.isnan(Var[:,j+1]))).size == 0:
            Var_z[:,j+1] = np.gradient(Var[:,j+1], Z)        
        else:
            zn = np.where(np.isnan(Var[:,j+1]))[-1][0] # Upper boundary of the mask
            Var_z[:zn,j+1] = np.gradient(Var[:zn,j+1], Z[:zn])
            
    return (Var_x, Var_z)
#L = np.where(np.isnan(us))[-1][0]
#xn1[23], xn2[23], u[23,:66]
#np.where(np.isnan(u[22,1:]))


In [33]:
u_x, u_z = masked_gradient(u, x, z)

uma[:,71], u_z[:,71], np.gradient(u[:,71],z)
#np.full((3, 5), np.nan)

(masked_array(data=[0.2438339740037918, 0.24162590503692627,
                    0.23957189917564392, 0.23863622546195984,
                    0.23889364302158356, 0.2395806610584259,
                    0.23966005444526672, 0.23818732798099518,
                    0.23527246713638306, 0.22923807799816132,
                    0.22015246748924255, 0.17671410739421844,
                    0.10184017568826675, 0.013444000855088234,
                    -0.08486006408929825, -0.1816128045320511,
                    -0.24896223843097687, -0.2766095995903015,
                    -0.2801664471626282, -0.2434195876121521, --, --, --,
                    --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                    --, --, --, --, --, --, --, --, --, --, --, --, --],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False, False, False, False, False,
                    False, False, False, False,  True,  True,  Tru

In [11]:
# Along-fjord velocity gradients
ux = np.gradient(uma, x, axis=1)
uz = np.gradient(uma, z, axis=0)
# Advective terms
Adv_h = uma * ux
Adv_v = wma * uz
advv = np.ma.masked_array(uz, mask)
# Viscous term
Visc = np.gradient(Avma*uz, z, axis=0) #(Az*Uz)z

g = 9.8
rhoz = np.gradient(rho, z, axis=0)
N2 = -(g/rho)*rhoz / (uz**2)

#Adv_h[mask] = np.nan
#Adv_v[mask] = np.nan
#Visc[mask] = np.nan

range1 = np.array([np.nanmax(Adv_h), np.nanmin(Adv_h), np.nanmean(Adv_h)])
range2 = np.array([np.nanmax(Adv_v), np.nanmin(Adv_v), np.nanmean(Adv_v)])
range3 = np.array([np.nanmax(Visc), np.nanmin(Visc), np.nanmean(Visc)])
#range1, range2, range3
ux[:,71], uz[:,71]

(masked_array(data=[0.00017880067753139883, 0.00017976279195863754,
                    0.00018104928312823176, 0.00018115311104338616,
                    0.00017742642376106232, 0.00016748346388339996,
                    0.00015074021939653903, 0.00012613956641871482,
                    8.530955528840423e-05, 1.3700804629479535e-05,
                    -0.00013522797962650657, -0.0005859435768797994,
                    -0.0007735802792012691, -0.0009281451930291951,
                    -0.0010209658648818731, -0.0009590163244865835,
                    7.892793655628338e-05, 0.0009525929344817996,
                    0.0013830260140821338, 0.0012834310764446855, --, --,
                    --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                    --, --, --, --, --, --, --, --, --, --, --, --, --, --],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False, False, False, False, False,
           

In [ ]:
fig1 = plt.figure(figsize=(16,10))
fig1.tight_layout()

cm = np.linspace(-1.2,1.2,20) # Contour range for momentum terms
cs = np.linspace(20,30,10)
cmap = 'RdBu_r'

Xl, Xr = 55, 85 # X range
Zu, Zl = 0, 30 # Z range

# Horizontal Advective Term Plot
ax1 = fig1.add_subplot(311)
im1 = ax1.contourf(x[Xl:Xr], z[Zu:Zl], Adv_h[Zu:Zl,Xl:Xr]*1e3, cm, cmap=cmap, extend='both')
ax1.xaxis.set_visible(False)
ax1.set_xlim([x[Xl], x[Xr-1]])
ax1.set_facecolor('k')

cbaxes = fig1.add_axes([0.75, 0.92, 0.15, 0.02])
cbar = fig1.colorbar(im1,cax=cbaxes, orientation='horizontal')
cbar.set_ticks([-2, -1, 0, 1, 2])
cbar.ax.set_title(r'$\times10^{-3}(m/s^2)$', size=12, weight='bold')
# Plot isohalines
CS1 = ax1.contour(x[Xl:Xr], z[Zu:Zl], sma[Zu:Zl,Xl:Xr], cs, colors = 'grey')
plt.clabel(CS1, fmt='%1.1f', fontsize=10, inline=1)
ax1.axvline(x=5800, color='g',lw=2)
ax1.set_ylabel('Depth (m)', size=15)
ax1.tick_params(labelsize=12)
ax1.set_title(r'$\mathbf{uu_x}$', loc='left', size=20)
ax1.set_title(r'$Q_{sg}=$'+str(Qsg)+r'$m^3/s,h_s=$'+str(hs)+'$m$', loc='center', size=20)



# Vertical Advective Term Plot
ax2 = fig1.add_subplot(312)
im2 = ax2.contourf(x[Xl:Xr], z[Zu:Zl], Adv_v[Zu:Zl,Xl:Xr]*1e3, cm, cmap=cmap, extend='both')
ax2.xaxis.set_visible(False)
ax2.set_xlim([x[Xl], x[Xr-1]])
ax2.set_facecolor('k')
# Plot isohalines
CS2 = ax2.contour(x[Xl:Xr], z[Zu:Zl], sma[Zu:Zl,Xl:Xr], cs, colors = 'grey')
plt.clabel(CS2, fmt='%1.1f', fontsize=10, inline=1)
ax2.axvline(x=5800, color='g',lw=2)
ax2.set_ylabel('Depth (m)', size=15)
ax2.tick_params(labelsize=12)
ax2.set_title(r'$\mathbf{wu_z}$', loc='left', size=20)



# qf-tide Plot
ax3 = fig1.add_subplot(313)
im3 = ax3.contourf(x[Xl:Xr], z[Zu:Zl], Visc[Zu:Zl,Xl:Xr]*1e3, cm, cmap=cmap, extend='both')
ax3.set_xlim([x[Xl], x[Xr-1]])
ax3.set_facecolor('k')
# Plot isohalines
CS3 = ax3.contour(x[Xl:Xr], z[Zu:Zl], sma[Zu:Zl,Xl:Xr], cs, colors = 'grey')
plt.clabel(CS3, fmt='%1.1f', fontsize=10, inline=1)
ax3.axvline(x=5800, color='g',lw=2)
ax3.set_ylabel('Depth (m)', size=15)
ax3.tick_params(labelsize=12)
ax3.set_title(r'$\mathbf{(A_v{u_z})_z}$', loc='left', size=20)
ax3.set_xlabel('Distance from glacier (m)', size=15)
ax3.tick_params(labelsize=12)


In [ ]:
x[55], x[72]

In [ ]:
def AF_Momentum(Qsg, hs):
    
    file1 = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf' + str(format(Qsg,'03d')) + '/state_' + str(format(hs,'03d')) + '.nc')
    file2 = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf' + str(format(Qsg,'03d')) + '/kpp_state_' + str(format(hs,'03d')) + '.nc')
    t0 = 81
    tn = len(file1.T)
    state1 = file1.isel(X=range(120), Xp1=range(121), Y=range(30,39), Yp1=range(30,40), T=range(t0,tn))
    state2 = file2.isel(X=range(120), Y=range(30,39), T=range(t0-1,tn-1))

    # U (T,Z,Y,Xp1) W(T,Zl,Y,X) KPPviscAz(T-1,Z,Y,X)
    U = state1.U.data.mean(axis=(0,2))
    u = (U[:, 1:] + U[:, :-1]) / 2
    W0 = state1.W.data.mean(axis=(0,2))
    Wbt = np.zeros((1,W0.shape[1])) # Add vertical velocity (0 m/s) at the bottom
    W = np.concatenate((W0, Wbt), axis=0)
    w = (W[1:, :] + W[:-1, :]) / 2
    #W = np.concatenate((W0, Wbt), axis=0)
    Av = state2.KPPviscAz.data.mean(axis=(0,2)) # Vetical eddy viscosity (m^2/s)
    s = state1.S.data.mean(axis=(0,2)) # Salinity
    x = state1.X.data
    z = state1.Z.data
    #file2.KPPviscAz
    #X, Z = np.meshgrid(x, z)
    mask = (s == 0) # Exclude data covered by the sill
    #s[mask] = np.nan
    uma = np.ma.masked_array(u,mask)
    wma = np.ma.masked_array(w,mask)
    sma = np.ma.masked_array(s,mask)
    Avma = np.ma.masked_array(Av,mask)
    
    # Along-fjord velocity gradients
    ux = np.gradient(uma, x, axis=1)
    uz = np.gradient(uma, z, axis=0)
    # Advective terms
    Adv_h = uma * ux
    Adv_v = wma * uz
    # Viscous term
    Visc = np.gradient(Avma*uz, z, axis=0) #(Az*Uz)z
    #Adv_h[mask] = np.nan
    #Adv_v[mask] = np.nan
    #Visc[mask] = np.nan      
    
    return (Adv_h, Adv_v, Visc)

In [ ]:
sd = np.array([9, 12, 15, 18, 30])
Qf = 700
X0, Xs = 55, 72 # X range for the sill slope

ADVh = np.empty(len(sd))
ADVv = np.empty(len(sd))
Vis = np.empty(len(sd))
for i in range(len(sd)):
    advh, advv, visc = AF_Momentum(Qf, sd[i])
    ADVh[i] = np.mean(advh[Zu:Zl, X0:Xs])
    ADVv[i] = np.mean(advv[Zu:Zl, X0:Xs])
    Vis[i] = np.mean(visc[Zu:Zl, X0:Xs])

ADVh, ADVv, Vis

In [ ]:
#labels = [r'$uu_x$', r'$wu_z$', r'$(A_v{u_z})_z$']
labels = [r'$h_s=9m$', r'$h_s=12m$', r'$h_s=15m$', r'$h_s=18m$', r'$h_s=30m$']


N = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars


fig2, ax = plt.subplots(figsize=(10,7))

ax.bar(N - width, np.abs(ADVh)*1e5, width, label=r'$|uu_x|$', facecolor='r', edgecolor='r', linewidth=3, alpha=.5)
ax.bar(N , np.abs(ADVv)*1e5, width, label=r'$|wu_z|$', facecolor='g', edgecolor='g', linewidth=3, alpha=.5)
ax.bar(N + width, np.abs(Vis)*1e5, width, label=r'$|(A_v{u_z})_z|$', facecolor='b', edgecolor='b', linewidth=3, alpha=.5)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel(r'$\times10^{-5}(m/s^2)$', fontsize=20)
#ax.set_title('Scores by group and gender')
ax.set_xticks(N)
ax.set_xticklabels(labels, fontsize=20)
ax.legend(fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
fig2.tight_layout()

In [ ]:
fig_path = "/Users/weiyangbao/Documents/Results_Figs/Parameter/"
if not os.path.exists(fig_path):
    os.makedirs(fig_path)

fig1.savefig(fig_path + '/Mom_dis.png', dpi=200, bbox_inches='tight')
fig2.savefig(fig_path + '/Mom_mag.png', dpi=200, bbox_inches='tight')